In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [1]:
!pip install radon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 1.5 MB/s eta 0:00:00


In [7]:
TXT_FILENAME = 'em_repo_stars_part3.txt'
with open(TXT_FILENAME) as f:
    repos = [line.strip() for line in f.readlines()]

In [8]:
import os
from fnmatch import fnmatch

def get_python_filenames(root_dir):
    filenames = []
    for path, subdirs, files in os.walk(root_dir):
        for name in files:
            if fnmatch(name, "*.py"):
                filenames.append(os.path.join(path, name))
    return filenames

In [ ]:
import subprocess
from tqdm.notebook import tqdm

pbar = tqdm(repos)
for repo in pbar:
    # get folder name
    folder_name = repo.split('/')[-1]
    # get repo name
    repo_name = "_".join(repo.split('/')[-2:])
    pbar.set_description_str(f"Processing {repo_name}")
    # clone repo
    pbar.set_postfix_str("Cloning Repository...")
    subprocess.run(["git", "clone", repo])
    # calculate metrics
    pbar.set_postfix_str(f"Running Radon...")
    full_filenames = get_python_filenames(folder_name)
    subprocess.run(["mkdir", "-p", f"Results/{repo_name}/Halstead/"])
    subprocess.run(["mkdir", "-p", f"Results/{repo_name}/MI/"])
    for i, filename in enumerate(tqdm(full_filenames)):
        with open(f"Results/{repo_name}/Halstead/results.jsonl", 'a') as f:
            subprocess.run(
                ["radon", "hal", "-f", filename, '-j'],
                stdout=f
            )
        with open(f"Results/{repo_name}/MI/results.jsonl", 'a') as f:
            subprocess.run(
                ["radon", "mi", filename, '-j'],
                stdout=f
            )
    # # save to gdrive
    pbar.set_postfix_str(f"Saving Results to GDrive...")
    subprocess.run(['zip', '-r', 'RadonResultsPart2_2.zip', 'Results'])
    subprocess.run(['cp', '-r', 'RadonResultsPart2_2.zip', 'drive/MyDrive'])
    subprocess.run(['rm', '-rf', 'RadonResultsPart2_2.zip'])
    # delete repository
    pbar.set_postfix_str(f"Deleting repository...")
    subprocess.run(['rm', '-rf', folder_name])